# Data Cleaning
delete files in manually labeled data (**release-level-data** from replication kit https://zenodo.org/record/5675024#.Ya-B8fHML0o) that are not related to bug fixes

In [1]:
import os

for filename in os.listdir('./release-level-data'):
    if filename.endswith("bug_fixes.json"):
        continue
    os.remove('./release-level-data/' + filename)
print("data cleaning done")

data cleaning done


# Extract traditional smells
returns a dictionary of list, with key represents a traditional smell and value represents list of classes that contain the smell

In [2]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # dictionary of list, key represents a traditional smell and value represents list of classes that contain the smell
        smell_classes_dict = {}
        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            smelly_classes = []
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smelly_classes.append(class_name)
            # map smell and classes containing the smell
            if smelly_classes:
                smell_name = filename.split(" ")[-1].split(".")[0]
                smell_classes_dict[smell_name] = smelly_classes

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_dict

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# Extract traditional smells test


In [3]:
smell_classes_dict = extract_traditional_smells('commons-bcel')
print(smell_classes_dict)

extracting traditional smells from repository: commons-bcel
Analysing commons-bcel...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
ERROR: extracting traditionial smells failed
None


Exception in thread "main" java.lang.RuntimeException: java.io.FileNotFoundException: The source does not exist commons-bcel
	at padl.creator.javafile.eclipse.CompleteJavaFileCreator.<init>(CompleteJavaFileCreator.java:86)
	at sad.detection.generators.SmellDetectionHelper.analyseCodeLevelModelFromJavaSourceFilesEclipse(SmellDetectionHelper.java:523)
	at sad.detection.generators.SmellDetectionHelper.analyseCodeLevelModelFromJavaSourceFilesEclipse(SmellDetectionHelper.java:496)
	at sad.detection.generators.SmellDetectionHelper.analyseCodeLevelModelFromJavaSourceFilesEclipse(SmellDetectionHelper.java:484)
	at sad.detection.generators.SmellDetectionCaller.main(SmellDetectionCaller.java:19)
Caused by: java.io.FileNotFoundException: The source does not exist commons-bcel
	at padl.creator.javafile.eclipse.CompleteJavaFileCreator.<init>(CompleteJavaFileCreator.java:87)
	... 4 more


# Extract traditional smells 2
returns a set of classes that contain a smell

In [4]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells2(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # set of classes that contain the smell
        smell_classes_set = set()

        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smell_classes_set.add(class_name)

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_set

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# RQ1 - Fisher exact test

In [5]:
from git import Repo
import json
from scipy.stats import fisher_exact

# clone repo
# repo = Repo.clone_from('https://github.com/apache/ant-ivy', 'ant-ivy')
repo = Repo('ant-ivy')

# checkout to specific release
# print(repo.tags)
repo.git.checkout('1.4.1')

smelly_classes = extract_traditional_smells2('ant-ivy')
print(smelly_classes)

# number of classes with at least one fixing and with at least one anti-pattern 
w_fix_w_anti = 0
# number of classes with at least one fixing and without any anti-patterns
w_fix_wo_anti = 0
# number of classes without any fixing and with at least one anti-pattern 
wo_fix_w_anti = 0
# number of classes without any fixing and without any anti-patterns
wo_fix_wo_anti = 0

f = open('./release-level-data/ant-ivy-1.4.1_bug_fixes.json')
data = json.load(f)
for line in data:
    filename = line['file'].split("/")[-1].split(".")[0]
    if line['bug_fixes']:
        if filename in smelly_classes:
            w_fix_w_anti +=1
        else:
            w_fix_wo_anti += 1
    else:
        if filename in smelly_classes:
            wo_fix_w_anti +=1
        else:
            wo_fix_wo_anti += 1

f.close()

# computer fisher exact test
data = [[w_fix_w_anti, wo_fix_w_anti],
         [w_fix_wo_anti, wo_fix_wo_anti]]

oddsratio, pvalue = fisher_exact(data)
print('\n')
print('Fisher exact test odds ratio: {:.4f}'.format(oddsratio))
print('Fisher exact test p-value: {:.4f}'.format(pvalue))

NoSuchPathError: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/ant-ivy

# RQ2 - Relation antipattern and fault-proneness

# Import de données

In [1]:
def getProjects():
    return {
    "ant-ivy": "https://github.com/apache/ant-ivy",
    "archiva": "https://github.com/apache/archiva",
    "calcite": "https://github.com/apache/calcite",
    "cayenne": "https://github.com/apache/cayenne",
    "commons-bcel": "https://github.com/apache/commons-bcel",
    "commons-beanutils": "https://github.com/apache/commons-beanutils",
    "commons-codec": "https://github.com/apache/commons-codec",
    "commons-collections": "https://github.com/apache/commons-collections",
    "commons-compress": "https://github.com/apache/commons-compress",
    "commons-configuration": "https://github.com/apache/commons-configuration",
    "commons-dbcp": "https://github.com/apache/commons-dbcp",
    "commons-digester": "https://github.com/apache/commons-digester",
    "commons-io": "https://github.com/apache/commons-io",
    "commons-jcs": "https://github.com/apache/commons-jcs",
    "commons-jexl": "https://github.com/apache/commons-jexl",
    "commons-lang": "https://github.com/apache/commons-lang",
    "commons-math": "https://github.com/apache/commons-math",
    "commons-net": "https://github.com/apache/commons-net",
    "commons-scxml": "https://github.com/apache/commons-scxml",
    "commons-validator": "https://github.com/apache/commons-validator",
    "commons-vfs": "https://github.com/apache/commons-vfs",
    "deltaspike": "https://github.com/apache/deltaspike",
    "eagle": "https://github.com/apache/eagle",
    "giraph": "https://github.com/apache/giraph",
    "gora": "https://github.com/apache/gora",
    "jspwiki": "https://github.com/apache/jspwiki",
    "knox": "https://github.com/apache/knox",
    "kylin": "https://github.com/apache/kylin",
    "lens": "https://github.com/apache/lens",
    "mahout": "https://github.com/apache/mahout",
    "manifoldcf": "https://github.com/apache/manifoldcf",
    "nutch": "https://github.com/apache/nutch",
    "opennlp": "https://github.com/apache/opennlp",
    "parquet-mr": "https://github.com/apache/parquet-mr",
    "santuario-java": "https://github.com/apache/santuario-java",
    "systemml": "https://github.com/apache/systemds",
    "tika": "https://github.com/apache/tika",
    "wss4j": "https://github.com/apache/ws-wss4j"
}

In [2]:
def getRelease(isCorrespondance = False):
    if(isCorrespondance):
        return {
   "ant-ivy":[
      "1.4.1",
      "2.0.0",
      "2.1.0",
      "2.2.0",
      "2.3.0",
      "2.4.0"
   ],
   "archiva":[
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "2.0.0",
      "2.1.0",
      "2.2.0"
   ],
   "calcite":[
      "1.8.0",
      "1.9.0",
      "1.10.0",
      "1.11.0",
      "1.12.0",
      "1.13.0",
      "1.14.0",
      "1.15.0"
   ],
   "cayenne":[
      "3.0.0",
      "3.1.0"
   ],
   "commons-bcel":[
      "5.2",
      "6.0",
      "6.1",
      "6.2"
   ],
   "commons-beanutils":[
      "1.7.0",
      "1.8.0",
      "1.9.0"
   ],
   "commons-codec":[
      "1.2",
      "1.3",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "1.9",
      "1.10"
   ],
   "commons-collections":[
      "1.0",
      "2.0",
      "2.1",
      "3.0",
      "3.1",
      "3.2",
      "3.3",
      "4.0",
      "4.1"
   ],
   "commons-compress":[
      "1.9",
      "1.10",
      "1.11",
      "1.12",
      "1.13",
      "1.14",
      "1.15",
      "1.16"
   ],
   "commons-configuration":[
      "1.4",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "1.9",
      "1.10",
      "2.2"
   ],
   "commons-dbcp":[
      "1.3",
      "1.4",
      "2.0",
      "2.1",
      "2.2.0",
      "2.3.0",
      "2.4.0",
      "2.5.0"
   ],
   "commons-digester":[
      "1.4",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "3.0",
      "3.1",
      "3.2"
   ],
   "commons-io":[
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "1.4",
      "2.0",
      "2.1",
      "2.2",
      "2.3",
      "2.4",
      "2.5"
   ],
   "commons-jcs":[
      "1.0",
      "1.1",
      "1.3",
      "2.0",
      "2.1",
      "2.2"
   ],
   "commons-jexl":[
      "2.0",
      "2.1"
   ],
   "commons-lang":[
      "2.0",
      "2.1",
      "2.2",
      "2.3",
      "2.4",
      "2.5",
      "2.6",
      "3.0",
      "3.1",
      "3.2",
      "3.3",
      "3.4",
      "3.5",
      "3.6",
      "3.7"
   ],
   "commons-math":[
      "1.0",
      "1.1",
      "1.2",
      "2.0",
      "2.1",
      "2.2",
      "3.0",
      "3.1",
      "3.2",
      "3.3",
      "3.4",
      "3.5"
   ],
   "commons-net":[
      "1.0.0",
      "1.1.0",
      "1.2.0",
      "1.3.0",
      "1.4.0",
      "2.0",
      "2.1",
      "2.2",
      "3.0",
      "3.1",
      "3.2",
      "3.3",
      "3.4",
      "3.5",
      "3.6"
   ],
   "commons-scxml":[
      "0.5",
      "0.6",
      "0.7",
      "0.8",
      "0.9"
   ],
   "commons-validator":[
      "1.1.0",
      "1.2.0",
      "1.3.0",
      "1.4.0",
      "1.5.0",
      "1.6.0"
   ],
   "commons-vfs":[
      "1.0",
      "2.0",
      "2.1",
      "2.2"
   ],
   "deltaspike":[
      "0.1",
      "0.2",
      "0.3",
      "0.4",
      "0.5",
      "0.6",
      "0.7",
      "1.0.0",
      "1.1.0",
      "1.2.0",
      "1.3.0",
      "1.4.0",
      "1.5.0",
      "1.6.0",
      "1.7.0",
      "1.8.0"
   ],
   "eagle":[
      "0.3.0",
      "0.4.0"
   ],
   "giraph":[
      "1.0.0",
      "1.1.0"
   ],
   "gora":[
      "0.1",
      "0.2",
      "0.3",
      "0.4",
      "0.5",
      "0.6"
   ],
   "jspwiki":[
      "1.4.0",
      "1.5.0",
      "1.6.0",
      "1.7.0",
      "1.8.0",
      "2.0.36",
      "2.2.19",
      "2.4.56",
      "2.6.0",
      "2.8.0",
      "2.9.0",
      "2.10.0"
   ],
   "knox":[
      "0.3.0",
      "0.4.0",
      "0.5.0",
      "0.6.0",
      "0.7.0",
      "0.8.0",
      "0.9.0",
      "0.10.0",
      "0.11.0",
      "0.12.0",
      "0.13.0",
      "0.14.0",
      "1.0.0"
   ],
   "kylin":[
      "0.6.1",
      "0.7.1",
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "1.5.0",
      "1.6.0",
      "2.0.0",
      "2.1.0",
      "2.2.0"
   ],
   "lens":[
      "2.6.0",
      "2.7.0"
   ],
   "mahout":[
      "0.1",
      "0.2",
      "0.3",
      "0.4",
      "0.5",
      "0.6",
      "0.7",
      "0.8",
      "0.9",
      "0.10.0",
      "0.11.0",
      "0.12.0"
   ],
   "manifoldcf":[
      "0.1",
      "0.2",
      "0.3",
      "0.4",
      "0.5",
      "0.6",
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "1.4",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "1.9",
      "1.10",
      "2.0",
      "2.1",
      "2.2",
      "2.3",
      "2.4",
      "2.5",
      "2.6",
      "2.7",
      "2.8",
      "2.9",
      "2.10"
   ],
   "nutch":[
      "0.7",
      "0.8",
      "0.9",
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "1.4",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "1.9",
      "1.10",
      "1.11",
      "1.12",
      "1.13",
      "1.14",
      "2.0",
      "2.1",
      "2.2",
      "2.3"
   ],
   "opennlp":[
      "1.7.0",
      "1.8.0"
   ],
   "parquet-mr":[
      "1.0.0",
      "1.1.0",
      "1.2.0",
      "1.3.0",
      "1.4.0",
      "1.5.0",
      "1.6.0",
      "1.7.0",
      "1.8.0",
      "1.9.0"
   ],
   "santuario-java":[
      "1.4.5",
      "1.5.9",
      "2.0.0",
      "2.1.0"
   ],
   "systemml":[
      "0.9",
      "0.10",
      "0.11",
      "0.12",
      "0.13",
      "0.14",
      "0.15",
      "1.0.0",
      "1.1.0"
   ],
   "tika":[
      "0.1",
      "0.2",
      "0.3",
      "0.4",
      "0.5",
      "0.6",
      "0.7",
      "0.8",
      "0.9",
      "0.10",
      "1.0",
      "1.1",
      "1.2",
      "1.3",
      "1.4",
      "1.5",
      "1.6",
      "1.7",
      "1.8",
      "1.9",
      "1.10",
      "1.11",
      "1.12",
      "1.13",
      "1.14",
      "1.15",
      "1.16",
      "1.17"
   ],
   "wss4j":[
      "1.5.0",
      "1.6.0",
      "2.0.0",
      "2.1.0"
   ]
}
    else:
        return { 
    "ant-ivy": [
        "1.4.1",
        "2.0.0",
        "2.1.0",
        "2.2.0",
        "2.3.0",
        "2.4.0"
    ],
    "archiva": [
        "archiva-1.0",
        "archiva-1.1",
        "archiva-1.2",
        "archiva-1.3",
        "archiva-2.0.0",
        "archiva-2.1.0",
        "archiva-2.2.0"
    ],
    "calcite": [
        "calcite-1.8.0",
        "calcite-1.9.0",
        "calcite-1.10.0",
        "calcite-1.11.0",
        "calcite-1.12.0",
        "calcite-1.13.0",
        "calcite-1.14.0",
        "calcite-1.15.0"
    ],
    "cayenne": [
        "STABLE-3.0",
        "STABLE-3.1"
    ],
    "commons-bcel": [
        "BCEL_5_2",
        "BCEL_6_0",
        "BCEL_6_1",
        "BCEL_6_2"
    ],
    "commons-beanutils": [
        "BEANUTILS_1_7_0",
        "BEANUTILS_1_8_0",
        "BEANUTILS_1_9_0"
    ],
    "commons-codec": [
        "CODEC_1_2",
        "CODEC_1_3",
        "commons-codec-1.5",
        "1_6", 
        "1.7",
        "1.8",
        "1.9",
        "1.10"
    ],
    "commons-collections": [
        "collections-1.0",
        "collections-2.0",
        "collections-2.1",
        "collections-3.0",
        "collections-3.1",
        "collections-3.2",
        "collections-3.3",
        "collections-4.0",
        "collections-4.1"
    ],
    "commons-compress": [
        "rel/1.9",
        "rel/1.10",
        "rel/1.11",
        "rel/1.12",
        "rel/1.13",
        "rel/1.14",
        "rel/1.15",
        "rel/1.16"
    ],
    "commons-configuration": [
        "CONFIGURATION_1_4",
        "CONFIGURATION_1_5",
        "CONFIGURATION_1_6",
        "CONFIGURATION_1_7",
        "CONFIGURATION_1_8",
        "CONFIGURATION_1_9",
        "CONFIGURATION_1_10",
        "CONFIGURATION_2_2"
    ],
    "commons-dbcp": [
        "DBCP_1_3",
        "DBCP_1_4",
        "DBCP_2_0",
        "DBCP_2_1",
        "DBCP_2_2_0",
        "DBCP_2_3_0",
        "commons-dbcp-2.4.0",
        "commons-dbcp-2.5.0"
    ],
    "commons-digester": [
        "DIGESTER_1_4",
        "DIGESTER_1_5",
        "DIGESTER_1_6",
        "DIGESTER_1_7",
        "DIGESTER_1_8",
        "DIGESTER3_3_0",
        "DIGESTER3_3_1",
        "DIGESTER3_3_2"
    ],
    "commons-io": [
        "IO_1_0",
        "IO_1_1",
        "IO_1_2",
        "IO_1_3",
        "commons-io-1.4",
        "commons-io-2.0",
        "commons-io-2.1",
        "2.2",
        "2.3",
        "2.4",
        "2.5"
    ],
    "commons-jcs": [
        "JCS_1_0",
        "jcs_1_1_dev",
        "JCS_1_3",
        "commons-jcs-2.0",
        "commons-jcs-2.1",
        "commons-jcs-2.2"
    ],
    "commons-jexl": [
        "2.0",
        "2.1"
    ],
    "commons-lang": [
        "LANG_2_0",
        "LANG_2_1",
        "LANG_2_2",
        "LANG_2_3",
        "LANG_2_4",
        "LANG_2_5",
        "LANG_2_6",
        "LANG_3_0",
        "LANG_3_1",
        "LANG_3_2",
        "LANG_3_3",
        "LANG_3_4",
        "LANG_3_5",
        "LANG_3_7"
    ],
    "commons-math": [
        "MATH_1_0",
        "MATH_1_1",
        "MATH_1_2",
        "MATH_2_0",
        "MATH_2_1",
        "MATH_2_2",
        "MATH_3_0",
        "MATH_3_1",
        "MATH_3_2",
        "MATH_3_3",
        "MATH_3_4",
        "MATH_3_5"
    ],
    "commons-net": [
        "NET_1_0_0",
        "NET_1_1_0",
        "NET_1_2_0",
        "NET_1_3_0",
        "NET_1_4_0",
        "NET_2_0",
        "commons-net-2.1",
        "NET_2_2",
        "NET_3_0",
        "NET_3_1",
        "NET_3_2",
        "NET_3_3",
        "NET_3_4",
        "NET_3_5",
        "NET_3_6"
    ],
    "commons-scxml": [
        "SCXML_0_5",
        "SCXML_0_6",
        "SCXML_0_7",
        "SCXML_0_8",
        "SCXML_0_9"
    ],
    "commons-validator": [
        "VALIDATOR_1_1_0",
        "VALIDATOR_1_2_0",
        "VALIDATOR_1_3_0",
        "VALIDATOR_1_4_0",
        "VALIDATOR_1_5_0",
        "VALIDATOR_1_6"
    ],
    "commons-vfs": [
        "rel/commons-vfs-1.0",
        "rel/commons-vfs-2.0",
        "rel/commons-vfs-2.1",
        "rel/commons-vfs-2.2"
    ],
    "deltaspike": [
        "deltaspike-project-0.1-incubating",
        "deltaspike-project-0.2-incubating",
        "deltaspike-project-0.3-incubating",
        "deltaspike-project-0.4",
        "deltaspike-project-0.5",
        "deltaspike-project-0.6",
        "deltaspike-project-0.7",
        "deltaspike-project-1.0.0",
        "deltaspike-project-1.1.0",
        "deltaspike-project-1.2.0",
        "deltaspike-root-1.3.0",
        "deltaspike-1.4.0",
        "deltaspike-1.5.0",
        "deltaspike-1.6.0",
        "deltaspike-1.7.0",
        "deltaspike-1.8.0"
    ],
    "eagle": [
        "v0.3.0-incubating",
        "v0.4.0-incubating"
    ],
    "giraph": [
        "release-1.0.0-RC3",
        "release-1.1.0"
    ],
    "gora": [
        "0.1-incubating",
        "gora-0.2",
        "apache-gora-0.3",
        "apache-gora-0.4",
        "apache-gora-0.5",
        "apache-gora-0.6"
    ],
    "jspwiki": [
        "jspwiki_1_4_0",
        "jspwiki_1_5_0",
        "jspwiki_1_6_0",
        "jspwiki_1_7_0",
        "jspwiki_1_8_0",
        "jspwiki_2_0_36",
        "jspwiki_2_2_19",
        "jspwiki_2_4_56",
        "jspwiki_2_6_0",
        "jspwiki_2_8_0",
        "jspwiki_2_9_0_rc1",
        "jspwiki_2_10_0"
    ],
    "knox": [
        "v0.3.0-final",
        "v0.4.0-release",
        "v0.5.0-rc2",
        "v0.6.0-release",
        "v0.7.0-release",
        "v0.8.0-release",
        "v0.9.0-release",
        "v0.10.0-release",
        "v0.11.0-release",
        "v0.12.0-release",
        "v0.13.0-release",
        "v0.14.0-release",
        "v1.0.0-release"
    ],
    "kylin": [
        "v0.6.1",
        "kylin-0.7.1-incubating",
        "kylin-1.0-incubating",
        "kylin-1.1-incubating",
        "kylin-1.2",
        "kylin-1.3",
        "kylin-1.5.0",
        "kylin-1.6.0",
        "kylin-2.0.0",
        "kylin-2.1.0",
        "kylin-2.2.0"
    ],
    "lens": [
        "apache-lens-2.6.0",
        "apache-lens-2.7.0"
    ],
    "mahout": [
        "mahout-0.1",
        "mahout-0.2",
        "mahout-0.3",
        "mahout-0.4",
        "mahout-0.5",
        "mahout-0.6",
        "mahout-0.7",
        "mahout-0.8",
        "mahout-0.9",
        "mahout-0.10.0",
        "mahout-0.11.0",
        "mahout-0.12.0"
    ],
    "manifoldcf": [
        "release-0.1-incubating",
        "release-0.2-incubating",
        "release-0.3-incubating",
        "release-0.4-incubating",
        "release-0.5-incubating",
        "release-0.6",
        "release-1.0",
        "release-1.1",
        "release-1.2",
        "release-1.3",
        "release-1.4",
        "release-1.5",
        "release-1.6",
        "release-1.7",
        "release-1.8",
        "release-1.9",
        "release-1.10",
        "release-2.0",
        "release-2.1",
        "release-2.2",
        "release-2.3",
        "release-2.4-RC0",
        "release-2.5",
        "release-2.6",
        "release-2.7",
        "release-2.8",
        "release-2.9",
        "release-2.10"
    ],
    "nutch": [
        "release-0.7",
        "release-0.8",
        "release-0.9",
        "release-1.0",
        "relase-1.1",
        "release-1.2",
        "release-1.3",
        "release-1.4",
        "release-1.5",
        "release-1.6",
        "release-1.7",
        "release-1.8",
        "release-1.9",
        "release-1.10",
        "release-1.11-rc2",
        "release-1.12",
        "release-1.13",
        "release-1.14",
        "release-2.0",
        "release-2.1",
        "release-2.2",
        "release-2.3"
    ],
    "opennlp": [
        "opennlp-1.7.0",
        "opennlp-1.8.0"
    ],
    "parquet-mr": [
        "parquet-1.0.0",
        "parquet-1.1.0",
        "parquet-1.2.0",
        "parquet-1.3.0",
        "parquet-1.4.0",
        "parquet-1.5.0",
        "apache-parquet-mr-1.6.0-incubating",
        "apache-parquet-1.7.0",
        "apache-parquet-1.8.0",
        "apache-parquet-1.9.0"
    ],
    "santuario-java": [
        "1.4.5",
        "xmlsec-1.5.8",
        "xmlsec-2.0.0",
        "xmlsec-2.1.0"
    ],
    "systemml": [
        "v0.9.0-rc1",
        "0.10.0-incubating-rc1",
        "v0.11.0-incubating-rc1",
        "v0.12.0-incubating-rc1",
        "v0.13.0-incubating-rc1",
        "v0.14.0-incubating-rc1",
        "v0.15.0-rc1",
        "v1.0.0-rc1",
        "v1.1.0-rc1"
    ],
    "tika": [
        "0.1-incubating",
        "0.2",
        "0.3",
        "0.4",
        "0.5",
        "0.6",
        "0.7",
        "0.8",
        "0.9",
        "0.10",
        "1.0",
        "1.1",
        "1.2",
        "1.3",
        "1.4",
        "1.5",
        "1.6",
        "1.7",
        "1.8",
        "1.9-rc2",
        "1.10",
        "1.11",
        "1.12",
        "1.13",
        "1.14",
        "1.15",
        "1.16",
        "1.17"
    ],
    "wss4j": [
        "1_5_0",
        "1_6_0",
        "wss4j-2.0.0",
        "wss4j-2.1.0"
    ]
}

      
        
def getReleaseCorr(projectName, release):
    pos = getRelease()[projectName].index(release)
    return getRelease(True)[projectName][pos]
        

In [17]:
from git import Repo
import os

# clone repos
for project_name, git_url in getProjects().items():
    if not os.path.isdir("./"+project_name):
        print(git_url)
        print(project_name)
        repo = Repo.clone_from(git_url, project_name)
print("... Done!")

https://github.com/apache/commons-io
commons-io
https://github.com/apache/commons-jcs
commons-jcs
https://github.com/apache/commons-jexl
commons-jexl
https://github.com/apache/commons-lang
commons-lang
https://github.com/apache/commons-math
commons-math
https://github.com/apache/commons-net
commons-net
https://github.com/apache/commons-scxml
commons-scxml
https://github.com/apache/commons-validator
commons-validator
https://github.com/apache/commons-vfs
commons-vfs
https://github.com/apache/deltaspike
deltaspike
https://github.com/apache/eagle
eagle
https://github.com/apache/giraph
giraph
https://github.com/apache/gora
gora
https://github.com/apache/jspwiki
jspwiki
https://github.com/apache/knox
knox
https://github.com/apache/kylin
kylin
https://github.com/apache/lens
lens
https://github.com/apache/mahout
mahout
https://github.com/apache/manifoldcf
manifoldcf
https://github.com/apache/nutch
nutch
https://github.com/apache/opennlp
opennlp
https://github.com/apache/parquet-mr
parquet-mr


# Données automatiques

In [29]:
from git import Repo
from math import exp
import json


# clone repo
# repo = Repo.clone_from('https://github.com/apache/ant-ivy', projetName)

# Open result file
f = open('result_b.txt', 'w')
    
for project_name, git_url in getProjects().items():
    # projet name
    print('\n')
    print("===+> projet name", project_name)
    print('\n')
    repo = Repo(project_name)

    # Seuil en dessous duquel il n'y a pas correlation
    seuil = 0.75

    # Write project name in file
    f.write(project_name)
    f.write('\n')

    # checkout to specific release
    # print('\n')
    # print('-- TAGS --')
    # print(repo.tags)

    # dictionnaire des correlation de chaque smell avec les classes
    smell_correlation_dict = {}

    tags = getRelease()[project_name]
    for tag in tags:
        print('\n')
        print(tag)

        repo.git.checkout(tag)

        # smelly_classes = extract_traditional_smells2('ant-ivy')
        # print('\n')
        # print('-- Smelly classes --')
        # print(smelly_classes)

        smell_classes_dict = extract_traditional_smells(project_name)
        # print('-- Smell in classes --')
        # print(smell_classes_dict)
        # print('\n')


        for smell, classes in smell_classes_dict.items():
            size = len(classes)
            size_exp = exp(size)
            # print("size : ",size)
            # print("size_exp", size_exp)
            correlation_value = (size_exp)/(1+size_exp)

            if correlation_value > seuil:
                # print(smell, " : ", correlation_value)
                smell_correlation_dict[smell] = smell_correlation_dict.get(smell, 0) +1



    f.write(str(smell_correlation_dict))
    f.write('\n')
    print(smell_correlation_dict)

f.close()
print("... Done!")



===+> projet name systemml




v0.9.0-rc1
extracting traditional smells from repository: systemml
Analysing systemml...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
Model built in 9633 ms.
Model contains 3074 top-level entities.
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current entity: org.apache.sys

0 solutions for Blob in systemml in 326 ms.
86 solutions for ClassDataShouldBePrivate in systemml in 10 ms.
366 solutions for ComplexClass in systemml in 188 ms.
0 solutions for FunctionalDecomposition in systemml in 187 ms.
0 solutions for LargeClass in systemml in 8 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composi

0 solutions for Blob in systemml in 317 ms.
80 solutions for ClassDataShouldBePrivate in systemml in 7 ms.
406 solutions for ComplexClass in systemml in 88 ms.
0 solutions for FunctionalDecomposition in systemml in 170 ms.
0 solutions for LargeClass in systemml in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite

Composite Code Smell has no main class.
1 solutions for Blob in systemml in 358 ms.
85 solutions for ClassDataShouldBePrivate in systemml in 9 ms.
407 solutions for ComplexClass in systemml in 99 ms.
0 solutions for FunctionalDecomposition in systemml in 200 ms.
0 solutions for LargeClass in systemml in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite

Composite Code Smell has no main class.
1 solutions for Blob in systemml in 534 ms.
87 solutions for ClassDataShouldBePrivate in systemml in 17 ms.
413 solutions for ComplexClass in systemml in 153 ms.
0 solutions for FunctionalDecomposition in systemml in 213 ms.
0 solutions for LargeClass in systemml in 22 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Compos

...

0 solutions for FunctionalDecomposition in tika in 86 ms.
0 solutions for LargeClass in tika in 5 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
9 solutions for LazyClass in tika in 13 ms.
230 solutions for LongMethod in tika in 17 ms.
19 solutions for LongParameterList in tika in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for ManyFieldAttributesButNotComplex in tika in 7 ms.
0 solutions for MessageChains in tika in 7 ms.
0 solutions for RefusedParentBequest in tika in 55 ms.
0 solutions for SpaghettiCode in tika in 14 ms.
0 solutions for SpeculativeGenerality in tika in 3 ms.
0 solutions for SwissArmyKnife in tika in 1 

extracting traditional smells from repository: wss4j
Analysing wss4j...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 1822 ms.
Model contains 628 top-level entities.
4 solutions for AntiSingleton in wss4j in 5 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a runtime deprecation: call

Traditional smells extracted for project:  wss4j


wss4j-2.0.0
extracting traditional smells from repository: wss4j
Analysing wss4j...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 4632 ms.
Model contains 2072 top-level entities.
1 solutions for AntiSingleton in wss4j in 5 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
uti

137 solutions for ComplexClass in wss4j in 64 ms.
0 solutions for FunctionalDecomposition in wss4j in 62 ms.
0 solutions for LargeClass in wss4j in 4 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
11 solutions for LazyClass in wss4j in 8 ms.
176 solutions for LongMethod in wss4j in 16 ms.
54 solutions for LongParameterList in wss4j in 7 ms.
0 solutions for ManyFieldAttributesButNotComplex in wss4j in 7 ms.
0 solutions for MessageChains in wss4j in 7 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for RefusedParentBequest in wss4j in 33 ms.
0 solutio

132 solutions for ComplexClass in wss4j in 42 ms.
0 solutions for FunctionalDecomposition in wss4j in 54 ms.
0 solutions for LargeClass in wss4j in 3 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
11 solutions for LazyClass in wss4j in 7 ms.
187 solutions for LongMethod in wss4j in 18 ms.
58 solutions for LongParameterList in wss4j in 6 ms.
0 solutions for ManyFieldAttributesButNotComplex in wss4j in 5 ms.
0 solutions for MessageChains in wss4j in 5 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for RefusedParentBequest in wss4j in 29 ms.
0 solutio

# Données manuelles et openSSZ

In [30]:
from git import Repo



# Seuil en dessous duquel il n'y a pas correlation
seuil = 0.75

# Open result file
ff = open('result2_b.txt', 'w')

for project_name, git_url in getProjects().items():

    # dictionnaire des correlation de chaque smell avec les classes
    smell_correlation_dict = {}
    
    # projet name
    print('\n')
    print("===+> projet name", project_name)
    print('\n')
    repo = Repo(project_name)

    
    # Write project name in file
    ff.write(project_name)
    ff.write('\n')
    
    # checkout to specific release
    tags = getRelease()[project_name]
    for tag in tags:
        
        # dictionnaire des smells
        manu_smell_dict = {}
        
        tagCorr = getReleaseCorr(project_name, tag)
        print('\n')
        print('tag : ', tag)
        print('tag corr : ', tagCorr)
        
        # checkout to specific release
        repo.git.checkout(tag)

        smelly_classes = extract_traditional_smells(project_name)



        f = open('./release-level-data/'+project_name+'-'+tagCorr+'_bug_fixes.json')
        data = json.load(f)
        for line in data:
            filename = line['file'].split("/")[-1].split(".")[0]
            if line['bug_fixes']:
                #print(filename)

                for smell, classes in smelly_classes.items():
                    if filename in classes:
                        manu_smell_dict.setdefault(smell, []).append(filename)


        print(manu_smell_dict)

        for smell, classes in manu_smell_dict.items():
            size = len(classes)
            size_exp = exp(size)
            print("size : ",size)
            print("size_exp", size_exp)
            correlation_value = (size_exp)/(1+size_exp)

            if correlation_value > seuil:
                print(smell, " : ", correlation_value)
                smell_correlation_dict[smell] = smell_correlation_dict.get(smell, 0) +1
        f.close()
        
    print('\n')
    print('\n')
    print('--- Resultats ---')
    ff.write(str(smell_correlation_dict))
    ff.write('\n')
    print(smell_correlation_dict)
ff.close()
print("... Done!")



===+> projet name systemml




tag :  v0.9.0-rc1
tag corr :  0.9
extracting traditional smells from repository: systemml
Analysing systemml...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
Model built in 8391 ms.
Model contains 3074 top-level entities.
visited 1000 entities, current entity: org.apache.sysml.runtime.matrix.sort.CompactDoubleIntInputFormat
visited 1000 entities, current

0 solutions for Blob in systemml in 280 ms.
86 solutions for ClassDataShouldBePrivate in systemml in 9 ms.
366 solutions for ComplexClass in systemml in 91 ms.
0 solutions for FunctionalDecomposition in systemml in 134 ms.
0 solutions for LargeClass in systemml in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite

0 solutions for Blob in systemml in 317 ms.
80 solutions for ClassDataShouldBePrivate in systemml in 9 ms.
406 solutions for ComplexClass in systemml in 84 ms.
0 solutions for FunctionalDecomposition in systemml in 149 ms.
0 solutions for LargeClass in systemml in 8 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite

Composite Code Smell has no main class.
1 solutions for Blob in systemml in 362 ms.
85 solutions for ClassDataShouldBePrivate in systemml in 9 ms.
407 solutions for ComplexClass in systemml in 103 ms.
0 solutions for FunctionalDecomposition in systemml in 160 ms.
0 solutions for LargeClass in systemml in 8 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composit

Composite Code Smell has no main class.
1 solutions for Blob in systemml in 354 ms.
87 solutions for ClassDataShouldBePrivate in systemml in 8 ms.
413 solutions for ComplexClass in systemml in 106 ms.
0 solutions for FunctionalDecomposition in systemml in 207 ms.
0 solutions for LargeClass in systemml in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composit

...

0 solutions for FunctionalDecomposition in tika in 77 ms.
0 solutions for LargeClass in tika in 5 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
9 solutions for LazyClass in tika in 8 ms.
230 solutions for LongMethod in tika in 17 ms.
19 solutions for LongParameterList in tika in 5 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for ManyFieldAttributesButNotComplex in tika in 6 ms.
0 solutions for MessageChains in tika in 8 ms.
0 solutions for RefusedParentBequest in tika in 58 ms.
0 solutions for SpaghettiCode in tika in 14 ms.
0 solutions for SpeculativeGenerality in tika in 2 ms.
0 solutions for SwissArmyKnife in tika in 1 m

extracting traditional smells from repository: wss4j
Analysing wss4j...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 1842 ms.
Model contains 628 top-level entities.
4 solutions for AntiSingleton in wss4j in 5 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a runtime deprecation: call

extracting traditional smells from repository: wss4j
Analysing wss4j...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 4507 ms.
Model contains 2072 top-level entities.
1 solutions for AntiSingleton in wss4j in 5 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a runtime deprecation: cal

137 solutions for ComplexClass in wss4j in 54 ms.
0 solutions for FunctionalDecomposition in wss4j in 55 ms.
0 solutions for LargeClass in wss4j in 3 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
11 solutions for LazyClass in wss4j in 7 ms.
176 solutions for LongMethod in wss4j in 16 ms.
54 solutions for LongParameterList in wss4j in 6 ms.
0 solutions for ManyFieldAttributesButNotComplex in wss4j in 6 ms.
0 solutions for MessageChains in wss4j in 6 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for RefusedParentBequest in wss4j in 31 ms.
0 solutio

132 solutions for ComplexClass in wss4j in 50 ms.
0 solutions for FunctionalDecomposition in wss4j in 56 ms.
0 solutions for LargeClass in wss4j in 4 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
11 solutions for LazyClass in wss4j in 7 ms.
187 solutions for LongMethod in wss4j in 15 ms.
58 solutions for LongParameterList in wss4j in 8 ms.
0 solutions for ManyFieldAttributesButNotComplex in wss4j in 4 ms.
0 solutions for MessageChains in wss4j in 7 ms.
Composite Code Smell has no main class.
Composite Code Smell has no main class.
2 solutions for RefusedParentBequest in wss4j in 28 ms.
0 solutio

# Nettoyage de données

In [ ]:
import os
import shutil

# clone repos
for project_name, git_url in getProjects().items():
    if os.path.isdir("./"+project_name):
        print(project_name)
        shutil.rmtree("./"+project_name)
print("... Done!")